# Лабораторная работа 4. Линейная регрессия
Вариант 32

Выполнил Шульга Артём группа P33111

In [22]:
import pandas as pd

data = pd.read_csv("california_housing_train.csv")
data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
...,...,...,...,...,...,...,...,...,...
16995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0
16996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0
16997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0


### Статистика по датасету
Выведем имя колонки, количество элементов, среднее значение, среднее отклонение, минимальное, максимальное значение и квантили 0,25, 0,75, 0,9 соответственно

In [23]:
data.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


### Предобработка данных
Так как у нас отсутствуют NaN и иные виды пустых значений (согласно описанию датасета), а также отсутствуют категориальные признаки, то мы просто нормализуем наши данные

In [34]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_data = data.drop(['median_house_value'], axis=1)
y_data = data['median_house_value']

X_data = scaler.fit_transform(X_data)

Напишем методы для обучения, для использования модели (набора коэффициентов) и для её оценки

In [35]:
from sklearn.model_selection import train_test_split
import numpy as np

def fit(X, y):
    X = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)
    return np.linalg.solve(np.dot(X.T, X), np.dot(X.T, y))

def predict(X, w):
    X = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)
    return np.dot(X, w)

def score(y_predicted, y):
    d1 = y - y_predicted
    d2 = y - y.mean()
    return 1 - d1.dot(d1) / d2.dot(d2)

Составим модель, где возьмем все признаки

In [38]:
RANDOM_STATE = 2
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=RANDOM_STATE)
X_train = np.array(X_train)
y_train = np.array(y_train)

w = fit(X_train, y_train)

score(predict(X_test, w), y_test)

0.652276197318302

Предположим, что точные координаты дома не сильно влияют на целевую функцию

In [41]:
RANDOM_STATE = 2
X_data = data.drop(['median_house_value', 'longitude', 'latitude'], axis=1)
y_data = data['median_house_value']

X_data = scaler.fit_transform(X_data)

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=RANDOM_STATE)

X_train = np.array(X_train)
y_train = np.array(y_train)

w = fit(X_train, y_train)

score(predict(X_test, w), y_test)

0.5799897040119439

Попробуем убрать средний доход домохозяйства из параметров

In [42]:
RANDOM_STATE = 2
X_data = data.drop(['median_house_value', 'median_income'], axis=1)
y_data = data['median_house_value']

X_data = scaler.fit_transform(X_data)

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=RANDOM_STATE)

X_train = np.array(X_train)
y_train = np.array(y_train)

w = fit(X_train, y_train)

score(predict(X_test, w), y_test)

0.3871420795478019

: 

# Выводы
В случае, когда мы отбрасывали какие-либо признаки наблюдается резкое падение нашей оценки модели. В случае же применения всех данных нам параметров мы наблюдаем максимально возможные показатели. Вероятно требуется использовать все имеющиеся признаки и стараться не отбрасывать их, а заниматься каким-либо их упрощением